In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import BertModel
from TorchCRF import CRF



class BertCRFModel(nn.Module):
    def __init__(self, config: dict, intent_labels: list[str], slot_labels: list[str], dropout: float = 0.1):
        super(BertCRFModel, self).__init__()
        self.config = config
        self.intent_labels = intent_labels
        self.slot_labels = slot_labels
        self.dropout_rate = dropout
        
        # Load pre-trained BERT
        self.bert = BertModel.from_pretrained(self.config["model_dir"])
        
        # Dropout layer
        self.dropout = nn.Dropout(self.dropout_rate)
        
        # Intent classification layers
        self.intent_layer_1 = nn.Linear(self.bert.config.hidden_size, self.config["out_first_layer"])
        self.intent_activation_1 = nn.ReLU()
        self.intent_layer_2 = nn.Linear(self.config["out_first_layer"], len(self.intent_labels))
        
        # Slot filling layers
        self.slot_layer_1 = nn.Linear(self.bert.config.hidden_size, self.config["out_first_layer"])
        self.slot_activation_1 = nn.ReLU()
        self.slot_layer_2 = nn.Linear(self.config["out_first_layer"], len(self.slot_labels))
        
        # CRF layer for slot filling
        self.crf = CRF(len(self.slot_labels), batch_first = True)

    def forward(self, input_ids: torch.Tensor, token_type_ids: torch.Tensor, attention_mask: torch.Tensor, labels: torch.Tensor = None):
        # BERT outputs
        bert_outputs = self.bert(input_ids=input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask)
        sequence_output = bert_outputs.last_hidden_state  # Sequence output for slot filling
        pooled_output = bert_outputs.pooler_output       # Pooled output for intent classification
        
        # Intent classification
        intent_output_dropout = self.dropout(pooled_output)
        intent_out_layer_1 = self.intent_layer_1(intent_output_dropout)
        intent_act_1 = self.intent_activation_1(intent_out_layer_1)
        intent_logits = self.intent_layer_2(intent_act_1)
        
        # Slot filling
        slot_output_dropout = self.dropout(sequence_output)
        slot_out_layer_1 = self.slot_layer_1(slot_output_dropout)
        slot_act_1 = self.slot_activation_1(slot_out_layer_1)
        slot_logits = self.slot_layer_2(slot_act_1)
        
        if labels is not None:
            # CRF loss
            loss = -self.crf(slot_logits, labels, mask=attention_mask.bool(), reduction='mean')
            return loss, intent_logits
        else:
            # CRF decoding
            slot_predictions = self.crf.decode(slot_logits, mask=attention_mask.bool())
            return slot_predictions, intent_logits

# Example usage
config = {
    "model_dir": "bert-base-uncased",
    "out_first_layer": 768
}
intent_labels = ["intent1", "intent2", "intent3"]
slot_labels = ["slot1", "slot2", "slot3", "slot4"]

model = BertCRFModel(config, intent_labels, slot_labels)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
import torch

# Example input configuration
batch_size = 2
seq_length = 10  # Adjust this as needed
num_intent_labels = len(intent_labels)
num_slot_labels = len(slot_labels)

# Dummy input tensors
input_ids = torch.randint(0, 30522, (batch_size, seq_length))  # Random integers representing token ids
token_type_ids = torch.zeros((batch_size, seq_length), dtype=torch.long)  # All zeros (single sequence)
attention_mask = torch.ones((batch_size, seq_length), dtype=torch.long)  # All ones (no padding)

# Dummy labels for slot filling
labels = torch.randint(0, num_slot_labels, (batch_size, seq_length))

# Create the model instance
model = BertCRFModel(config, intent_labels, slot_labels)

# Put the model in evaluation mode (not strictly necessary for this test)
model.eval()

# Test the forward method with dummy inputs
with torch.no_grad():
    # Without labels (inference mode)
    slot_predictions, intent_logits = model(input_ids, token_type_ids, attention_mask)
    print("Slot Predictions (Inference):", slot_predictions)
    print("Intent Logits (Inference):", intent_logits)

    # With labels (training mode)
    loss, intent_logits = model(input_ids, token_type_ids, attention_mask, labels)
    print("Loss (Training):", loss)
    print("Intent Logits (Training):", intent_logits)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Slot Predictions (Inference): [[1, 1, 0, 0, 0, 0, 1, 0, 3, 3], [3, 1, 1, 1, 1, 1, 1, 0, 1, 1]]
Intent Logits (Inference): tensor([[-0.3224,  0.1625, -0.1054],
        [-0.2655,  0.1444, -0.1160]])
Loss (Training): tensor(13.9148)
Intent Logits (Training): tensor([[-0.3224,  0.1625, -0.1054],
        [-0.2655,  0.1444, -0.1160]])


In [1]:
# from bert_model_implementation_torch.model import BertModel
from bert_model_implementation_torch.tokenization import  _is_control,_is_whitespace,_is_punctuation
from transformers import BertTokenizer, BertModel
from sklearn.model_selection import train_test_split
import torch 
import torch.nn as nn
import numpy as np
import pandas as pd
import sys
from tqdm import tqdm
from keras.src.utils import to_categorical
from torch.utils.data import DataLoader
import shutil

d:\conda\Lib\site-packages\transformers\utils\generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
d:\conda\Lib\site-packages\transformers\utils\generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
05/18/2024 19:41:52 - INFO - numexpr.utils -   Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
05/18/2024 19:41:52 - INFO - numexpr.utils -   NumExpr defaulting to 8 threads.


In [9]:
data = pd.read_csv("../data/Book1.csv")

In [45]:
data.head()

,prompt,intent,classes
0,Can you update the sales figures in cells <Cel...,Update cell range,entry and manipulation
1,Please modify the data in range <Range>,Update cell range,entry and manipulation
2,I need to change the prices in cells <Range>,Update cell range,entry and manipulation
3,Could you update the inventory levels from <Ce...,Update cell range,entry and manipulation
4,Please adjust the budget numbers in range <Range>,Update cell range,entry and manipulation


In [46]:
config = {
    'max_len': 256,
    'batch_size': 8,
    'epochs': 10,
    'lr':1e-05,
    'out_first_layer': 768,
    'dropout_rate': 0.1,
    'model_dir':'bert-base-cased',
    'ckpt_path': './ckpts',
    'ckpt_model_path': './experiments'
}

In [10]:
intent_list = data.intent.unique().tolist()

In [48]:
class intent_dataset:
    def __init__(self, df: pd.DataFrame, tokenizer: BertTokenizer, max_len: int):
        self.df = df
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.classes = data.intent.unique().tolist()
        self.y = df['intent']
        self.x = df['prompt']
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        title = str(self.x[index])
        title= ''.join(title.split())
        inputs = self.tokenizer.encode_plus(
            title,
            None,
            add_special_tokens = True,
            return_attention_mask = True,
            return_tensors = 'pt',
            return_token_type_ids = True,
            padding = 'max_length',
            max_length = self.max_len,
            truncation = True
        )
        # print(self.classes.index(self.y[index]))
        target = self.classes.index(self.y[index])  # Get the class index
        target_tensor = torch.zeros(len(self.classes),dtype= torch.float32)  # Initialize target tensor with zeros
        target_tensor[target] = 1  # Set the corresponding index to 1
        return {
            'input_ids': inputs["input_ids"].flatten(),
            'token_type_ids': inputs['token_type_ids'].flatten(),
            'attention_mask': inputs['attention_mask'].flatten(),
            'targets': target_tensor
        }

In [49]:
train = data.sample(frac=0.9, random_state=200).reset_index(drop=True)
val = data.drop(train.index).reset_index(drop=True)

In [50]:
tokenizer = BertTokenizer.from_pretrained(config["model_dir"])
train_dataset = intent_dataset(train, tokenizer, config["max_len"])
val_dataset = intent_dataset(val, tokenizer, config["max_len"])

In [51]:
train_loader = DataLoader(
    train_dataset,
    shuffle = True,
    batch_size = config["batch_size"],
    num_workers = 0
    )

val_loader = DataLoader(
    val_dataset,
    shuffle = False,
    batch_size = config["batch_size"],
    num_workers = 0
    )

In [52]:
train_loader.dataset[10]['targets']


tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0.])

In [53]:
len(train_loader.dataset[0]['input_ids'])


256

In [12]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)

cuda


In [26]:
def load_ckpt(ckpt_path, model, optimizer):
    ckpt = torch.load(ckpt_path)
    # print(ckpt)
    model.load_state_dict(ckpt['state_dict'])
    # optimizer.load_state_dict(ckpt['state_dict'])
    valid_loss_min = ckpt['valid_loss_min']
    return model

In [3]:
def save_ckpt(state, is_best, ckpt_path, best_model_path):
    f_path= ckpt_path
    torch.save(state, f_path)
    if is_best:
        best_f_path = best_model_path
        shutil.copyfile(f_path, best_f_path)

In [7]:
class intent_model(nn.Module):
    def __init__(self, config: dict, intent_labels: list[str], dropout: float = 0.1):
        super(intent_model,self).__init__()
        self.config = config
        self.intent_labels = intent_labels
        self.dropout_rate = dropout
        self.bert = BertModel.from_pretrained(self.config["model_dir"])
        self.dropout = nn.Dropout(self.dropout_rate)
        self.layer_1 = nn.Linear(self.bert.config.hidden_size,self.config["out_first_layer"])
        self.activation_1 = nn.ReLU()
        self.layer_2 = nn.Linear(self.config["out_first_layer"], len(self.intent_labels))
        
    def forward(self, input_ids: torch.Tensor, token_type_ids: torch.Tensor, attention_mask: torch.Tensor):
        output = self.bert(input_ids, token_type_ids, attention_mask)
        output_dropout = self.dropout(output.pooler_output)
        out_layer_1 = self.layer_1(output_dropout)
        act_1 = self.activation_1(out_layer_1)
        out_layer_2 = self.layer_2(act_1)
        return out_layer_2
        

In [39]:
model = intent_model(config, intent_list, 0.1)
model.to(device)

05/16/2024 01:11:17 - INFO - bert_model_implementation_torch.file_utils -   https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased.tar.gz not found in cache, downloading to C:\Users\kikos\AppData\Local\Temp\tmpcjv867ym
100%|██████████| 404400730/404400730 [07:17<00:00, 925346.54B/s] 
05/16/2024 01:18:36 - INFO - bert_model_implementation_torch.file_utils -   copying C:\Users\kikos\AppData\Local\Temp\tmpcjv867ym to cache at C:\Users\kikos\.pytorch_pretrained_bert\a803ce83ca27fecf74c355673c434e51c265fb8a3e0e57ac62a80e38ba98d384.681017f415dfb33ec8d0e04fe51a619f3f01532ecea04edbfd48c5d160550d9c
05/16/2024 01:18:36 - INFO - bert_model_implementation_torch.file_utils -   creating metadata file for C:\Users\kikos\.pytorch_pretrained_bert\a803ce83ca27fecf74c355673c434e51c265fb8a3e0e57ac62a80e38ba98d384.681017f415dfb33ec8d0e04fe51a619f3f01532ecea04edbfd48c5d160550d9c
05/16/2024 01:18:36 - INFO - bert_model_implementation_torch.file_utils -   removing temp file C:\Users\kikos\AppData

intent_model(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
            )
          )
 

In [40]:
def loss_fn(outputs, targets):
    targets = targets.float()
    return nn.BCEWithLogitsLoss()(outputs, targets)


In [41]:
def train(model, epochs, train_loader, val_loader, optimizer, ckpt_path, best_model_path):
    valid_loss_min = np.Inf
    for epoch in range(1, epochs + 1):
        train_loss = 0
        valid_loss = 0
        model.train()
        for batch_index, batch in tqdm(enumerate(train_loader)):
            input_ids = batch['input_ids'].to(device, dtype= torch.long)
            attention_mask = batch["attention_mask"].to(device, dtype= torch.long)
            token_type_ids = batch["token_type_ids"].to(device, dtype= torch.long)
            targets = batch["targets"].to(device, dtype= torch.long)
            outputs = model(input_ids, token_type_ids, attention_mask)
            optimizer.zero_grad()
            loss = loss_fn(outputs, targets)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            train_loss += (1/(batch_index + 1)) * (loss.item() - train_loss)
        print(f"epoch {epoch} ended with train loss of {train_loss}")
        model.eval()
        with torch.no_grad():
            for batch_index, batch in tqdm(enumerate(val_loader)):
                input_ids = batch['input_ids'].to(device, dtype= torch.long)
                attention_mask = batch["attention_mask"].to(device, dtype= torch.long)
                token_type_ids = batch["token_type_ids"].to(device, dtype= torch.long)
                targets = batch["targets"].to(device, dtype= torch.long)
                outputs = model(input_ids, token_type_ids, attention_mask)
                loss = loss_fn(outputs, targets)
                valid_loss += (1/(batch_index + 1)) * (loss.item() - valid_loss)
        print(f"epoch {epoch} ended with train loss of {valid_loss}")
        checkpoint = {
            'epoch': epoch +1,
            'valid_loss_min': valid_loss,
            'state_dict': model.state_dict(),
            'optimizer': optimizer.state_dict()
        }
        save_ckpt(checkpoint, False, ckpt_path, best_model_path)
        return model

In [42]:
train = train(model, config["epochs"], train_loader, val_loader, optimizer, config["ckpt_path"], config["ckpt_model_path"])

0it [00:02, ?it/s]


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [33]:
config = {
    'max_len': 256,
    'batch_size': 8,
    'epochs': 10,
    'lr':1e-05,
    'out_first_layer': 768,
    'dropout_rate': 0.1,
    'model_dir':'bert-base-cased',
    'ckpt_path': './ckpts',
    'ckpt_model_path': './experiments'
}
import torch.nn.functional as F

In [54]:
class intent_model(nn.Module):
    def __init__(self, config: dict, intent_labels: list[str], dropout: float = 0.1):
        super(intent_model, self).__init__()
        self.config = config
        self.intent_labels = intent_labels
        self.dropout_rate = dropout
        self.bert = BertModel.from_pretrained(self.config["model_dir"])
        self.dropout = nn.Dropout(self.dropout_rate)
        self.layer_1 = nn.Linear(self.bert.config.hidden_size, self.config["out_first_layer"])
        self.activation_1 = nn.ReLU()
        self.layer_2 = nn.Linear(self.config["out_first_layer"], len(self.intent_labels))
        
    def forward(self, input_ids: torch.Tensor, token_type_ids: torch.Tensor, attention_mask: torch.Tensor):
        output = self.bert(input_ids, token_type_ids, attention_mask)
        output_dropout = self.dropout(output.pooler_output)
        out_layer_1 = self.layer_1(output_dropout)
        act_1 = self.activation_1(out_layer_1)
        out_layer_2 = self.layer_2(act_1)
        return out_layer_2
    
    def predict_intent(self, text: str, tokenizer: BertTokenizer):
        # Tokenize input text
        inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=128)
        input_ids = inputs['input_ids']
        token_type_ids = inputs['token_type_ids']
        attention_mask = inputs['attention_mask']
        
        # Move tensors to the appropriate device
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.to(device)
        input_ids = input_ids.to(device)
        token_type_ids = token_type_ids.to(device)
        attention_mask = attention_mask.to(device)
        
        # Make predictions
        self.eval()  # Set the model to evaluation mode
        with torch.no_grad():
            logits = self(input_ids, token_type_ids, attention_mask)
        
        # Convert logits to probabilities
        probs = F.softmax(logits, dim=1)
        
        # Get the predicted label
        predicted_label_idx = torch.argmax(probs, dim=1).item()
        print(predicted_label_idx)
        predicted_label = self.intent_labels[predicted_label_idx]
        
        return predicted_label, probs[0].cpu().numpy()

In [55]:
tokenizer = BertTokenizer.from_pretrained(config["model_dir"])

In [56]:
optimizer = torch.optim.Adam(params = model.parameters(), lr = config["lr"])

In [57]:
model = intent_model(config, intent_list, 0.1)
model.to(device)

intent_model(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affi

In [58]:
model.bert.resize_token_embeddings(30522)

You are resizing the embedding layer without providing a `pad_to_multiple_of` parameter. This means that the new embeding dimension will be 30522. This might induce some performance reduction as *Tensor Cores* will not be available. For more details  about this, or help on choosing the correct value for resizing, refer to this guide: https://docs.nvidia.com/deeplearning/performance/dl-performance-matrix-multiplication/index.html#requirements-tc


Embedding(30522, 768)

In [59]:
model = load_ckpt("./ckpts/ckpt", model, optimizer)

In [61]:
model.predict_intent("Please modify the data in range", tokenizer)

14


('Delete hyperlink',
 array([0.01483624, 0.00962736, 0.01309677, 0.01109273, 0.01267296,
        0.01391257, 0.01042866, 0.01172547, 0.01188861, 0.01360726,
        0.01299947, 0.01062253, 0.01138435, 0.01929716, 0.01973281,
        0.00815214, 0.01179272, 0.01155274, 0.01214933, 0.01029058,
        0.01119961, 0.00984573, 0.01025925, 0.01101724, 0.01066194,
        0.01142314, 0.01046023, 0.01088517, 0.01101524, 0.01014855,
        0.01382952, 0.00946699, 0.01382294, 0.01145752, 0.01230946,
        0.01451662, 0.0112405 , 0.0141823 , 0.00991062, 0.01725945,
        0.01911117, 0.0094124 , 0.0120395 , 0.00818533, 0.01099228,
        0.01047215, 0.00911422, 0.00737221, 0.01044737, 0.0114465 ,
        0.00703791, 0.00761131, 0.00768395, 0.00895203, 0.00710685,
        0.01233352, 0.00776994, 0.00646441, 0.00676312, 0.00951902,
        0.01012736, 0.01270906, 0.0093776 , 0.01227192, 0.01941985,
        0.011088  , 0.0120331 , 0.01338007, 0.00844096, 0.00996742,
        0.00847649, 0.00942

In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, BertModel, get_linear_schedule_with_warmup, AdamW
from tqdm import tqdm
import pandas as pd
import numpy as np
from TorchCRF import CRF


class BertCRFModel(nn.Module):
    def __init__(self, config: dict, intent_labels: list[str], slot_labels: list[str], dropout: float = 0.1):
        super(BertCRFModel, self).__init__()
        self.config = config
        self.intent_labels = intent_labels
        self.slot_labels = slot_labels
        self.dropout_rate = dropout
        
        # Load pre-trained BERT
        self.bert = BertModel.from_pretrained(self.config["model_dir"])
        
        # Dropout layer
        self.dropout = nn.Dropout(self.dropout_rate)
        
        # Intent classification layers
        self.intent_layer_1 = nn.Linear(self.bert.config.hidden_size, self.config["out_first_layer"])
        self.intent_activation_1 = nn.ReLU()
        self.intent_layer_2 = nn.Linear(self.config["out_first_layer"], len(self.intent_labels))
        
        # Slot filling layers
        self.slot_layer_1 = nn.Linear(self.bert.config.hidden_size, self.config["out_first_layer"])
        self.slot_activation_1 = nn.ReLU()
        self.slot_layer_2 = nn.Linear(self.config["out_first_layer"], len(self.slot_labels))
        
        # CRF layer for slot filling
        self.crf = CRF(len(self.slot_labels), batch_first=True)

    def forward(self, input_ids: torch.Tensor, token_type_ids: torch.Tensor, attention_mask: torch.Tensor, labels: torch.Tensor = None):
        # BERT outputs
        bert_outputs = self.bert(input_ids=input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask)
        sequence_output = bert_outputs.last_hidden_state  # Sequence output for slot filling
        pooled_output = bert_outputs.pooler_output       # Pooled output for intent classification
        
        # Intent classification
        intent_output_dropout = self.dropout(pooled_output)
        intent_out_layer_1 = self.intent_layer_1(intent_output_dropout)
        intent_act_1 = self.intent_activation_1(intent_out_layer_1)
        intent_logits = self.intent_layer_2(intent_act_1)
        
        # Slot filling
        slot_output_dropout = self.dropout(sequence_output)
        slot_out_layer_1 = self.slot_layer_1(slot_output_dropout)
        slot_act_1 = self.slot_activation_1(slot_out_layer_1)
        slot_logits = self.slot_layer_2(slot_act_1)
        
        if labels is not None:
            # CRF loss
            loss = -self.crf(slot_logits, labels, mask=attention_mask.bool(), reduction='mean')
            return loss, intent_logits
        else:
            # CRF decoding
            slot_predictions = self.crf.decode(slot_logits, mask=attention_mask.bool())
            return slot_predictions, intent_logits

# Dataset class
class isf_dataset(Dataset):
    def __init__(self, df: pd.DataFrame, intent_classes: list[str], slot_classes: list[str], tokenizer: BertTokenizer, max_len: int):
        self.df = df
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.intent_classes = intent_classes
        self.slot_classes = slot_classes
        self.intents = df['intent']
        self.slots = df["slots"]
        self.x = df['prompt']
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        title = str(self.x[index])
        title = ''.join(title.split())
        inputs = self.tokenizer.encode_plus(
            title,
            None,
            add_special_tokens=True,
            return_attention_mask=True,
            return_tensors='pt',
            return_token_type_ids=True,
            padding='max_length',
            max_length=self.max_len,
            truncation=True
        )
        intent_target = self.intent_classes.index(self.intents[index])
        slot_targets = [self.slot_classes.index(slot) for slot in list(self.slots[index])][:self.max_len] + [self.slot_classes.index("O")] * (self.max_len - len(list(self.slots[index])))
        return {
            'input_ids': inputs["input_ids"].flatten(),
            'token_type_ids': inputs['token_type_ids'].flatten(),
            'attention_mask': inputs['attention_mask'].flatten(),
            'intent_targets': torch.tensor(intent_target, dtype=torch.long),
            'slot_targets': torch.tensor(slot_targets, dtype=torch.long)
        }

def train_model(model, train_loader, val_loader, optimizer, scheduler, num_epochs, device):
    model.to(device)
    best_loss = float('inf')

    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        for batch in tqdm(train_loader, desc=f"Training Epoch {epoch + 1}"):
            input_ids = batch['input_ids'].to(device)
            token_type_ids = batch['token_type_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            intent_labels = batch['intent_targets'].to(device)
            slot_labels = batch['slot_targets'].to(device)

            optimizer.zero_grad()

            slot_loss, intent_logits = model(input_ids, token_type_ids, attention_mask, slot_labels)
            intent_loss = nn.CrossEntropyLoss()(intent_logits, intent_labels)
            loss = slot_loss + intent_loss

            loss.backward()
            optimizer.step()
            scheduler.step()

            total_loss += loss.item()

        avg_train_loss = total_loss / len(train_loader)
        print(f"Training loss: {avg_train_loss}")

        model.eval()
        total_val_loss = 0
        with torch.no_grad():
            for batch in tqdm(val_loader, desc="Validation"):
                input_ids = batch['input_ids'].to(device)
                token_type_ids = batch['token_type_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                intent_labels = batch['intent_targets'].to(device)
                slot_labels = batch['slot_targets'].to(device)

                slot_loss, intent_logits = model(input_ids, token_type_ids, attention_mask, slot_labels)
                intent_loss = nn.CrossEntropyLoss()(intent_logits, intent_labels)
                loss = slot_loss + intent_loss

                total_val_loss += loss.item()

        avg_val_loss = total_val_loss / len(val_loader)
        print(f"Validation loss: {avg_val_loss}")

        if avg_val_loss < best_loss:
            best_loss = avg_val_loss
            torch.save(model.state_dict(), "best_model.pt")
            print("Saved the best model!")
    return model

# Example usage
if __name__ == "__main__":
    # Configuration
    config = {
        "model_dir": "bert-base-uncased",
        "out_first_layer": 128,
        "dropout_rate": 0.1,
        "learning_rate": 1e-5,
        "num_epochs": 60,
        "batch_size": 16,
        "max_len": 128
    }
    intent_labels = ["intent1", "intent2", "intent3"]  # Example intents
    slot_labels = ["slot1", "slot2", "slot3", "O"]  # Example slots including "O" for outside of any slot

    # Initialize tokenizer
    tokenizer = BertTokenizer.from_pretrained(config["model_dir"])

    # Create dummy dataframe
    data = {
        "prompt": ["Example sentence one.", "Example sentence two."],
        "intent": ["intent1", "intent2"],
        "slots": [["O", "O", "slot1", "O"], ["O", "O", "slot2", "O"]]
    }
    df = pd.DataFrame(data)

    # Create datasets and dataloaders
    train_dataset = isf_dataset(df, intent_labels, slot_labels, tokenizer, max_len=config["max_len"])
    val_dataset = isf_dataset(df, intent_labels, slot_labels, tokenizer, max_len=config["max_len"])

    train_loader = DataLoader(train_dataset, batch_size=config["batch_size"], shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=config["batch_size"], shuffle=False)

    # Initialize model
    model = BertCRFModel(config, intent_labels, slot_labels)

    # Optimizer and scheduler
    optimizer = AdamW(model.parameters(), lr=config["learning_rate"])
    total_steps = len(train_loader) * config["num_epochs"]
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

    # Train the model
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = train_model(model, train_loader, val_loader, optimizer, scheduler, config["num_epochs"], device)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
d:\conda\Lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementatio

Training loss: 10.891477584838867


Validation: 100%|██████████| 1/1 [00:00<00:00,  8.25it/s]


Validation loss: 10.42165756225586
Saved the best model!


Training Epoch 2: 100%|██████████| 1/1 [00:00<00:00,  2.44it/s]


...

Validation loss: 3.1228561401367188
Saved the best model!


Training Epoch 60: 100%|██████████| 1/1 [00:00<00:00,  2.72it/s]


Training loss: 3.5584776401519775


Validation: 100%|██████████| 1/1 [00:00<00:00,  7.95it/s]


Validation loss: 3.120920181274414
Saved the best model!


In [12]:
import torch
from transformers import BertTokenizer
import numpy as np

def predict(model, tokenizer, prompt, intent_labels, slot_labels, max_len, device):
    model.eval()
    
    # Tokenize input
    inputs = tokenizer.encode_plus(
        prompt,
        None,
        add_special_tokens=True,
        return_attention_mask=True,
        return_tensors='pt',
        return_token_type_ids=True,
        padding='max_length',
        max_length=max_len,
        truncation=True
    )
    
    input_ids = inputs['input_ids'].to(device)
    token_type_ids = inputs['token_type_ids'].to(device)
    attention_mask = inputs['attention_mask'].to(device)
    
    with torch.no_grad():
        slot_predictions, intent_logits = model(input_ids, token_type_ids, attention_mask)
    
    # Get intent prediction
    intent_pred = torch.argmax(intent_logits, dim=1).item()
    intent_label = intent_labels[intent_pred]
    
    # Get slot predictions
    slot_predictions = slot_predictions[0]  # since batch size is 1
    slot_labels_pred = [slot_labels[slot] for slot in slot_predictions]
    
    # Decode token ids to words
    tokens = tokenizer.convert_ids_to_tokens(input_ids[0])
    
    # Filter out special tokens
    tokens = [token for token, mask in zip(tokens, attention_mask[0]) if mask == 1]
    slot_labels_pred = slot_labels_pred[:len(tokens)]
    
    return intent_label, list(zip(tokens, slot_labels_pred))

# Example usage
if __name__ == "__main__":
    # Assuming the model and tokenizer have been initialized as in the previous code
    model.load_state_dict(torch.load("best_model.pt"))
    model.to(device)

    # Example prompt
    prompt = "Book a flight from New York to Los Angeles next Monday."

    # Predict
    intent_label, token_slot_pairs = predict(model, tokenizer, prompt, intent_labels, slot_labels, config["max_len"], device)

    print(f"Intent: {intent_label}")
    print("Token - Slot pairs:")
    for token, slot in token_slot_pairs:
        print(f"{token}: {slot}")


Intent: intent1
Token - Slot pairs:
[CLS]: O
book: O
a: O
flight: O
from: O
new: O
york: O
to: O
los: O
angeles: O
next: O
monday: O
.: O
[SEP]: O
